In [77]:
#import necessary libraries
import folium
import pandas as pd

In [78]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [79]:
df = pd.read_csv("dataset_part_2.csv")
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


In [80]:
## Task 1: Mark all launch sites on a map
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [81]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
site_map

In [82]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Create and add folium.Circle and folium.Marker for each launch site on the site map

In [83]:
for i in range(0,3):
    latitude = launch_sites_df.loc[i]['Latitude']
    longitude = launch_sites_df.loc[i]['Longitude']
    coordinate = [latitude, longitude]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_sites_df.loc[i]['LaunchSite']))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coordinate,
    # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df.loc[i]['LaunchSite'],
         )
     )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [84]:
site_map

We can see that all launch sites in proximity to the Equator line.  And all launch sites in very close proximity to the coast.

In [85]:
## Task 2: Mark the success/failed launches for each site on the map
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class
80,CCSFS SLC 40,28.561857,-80.577366,1
81,CCSFS SLC 40,28.561857,-80.577366,1
82,CCSFS SLC 40,28.561857,-80.577366,1
83,CCSFS SLC 40,28.561857,-80.577366,1
84,CCSFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608058,-80.603956,1
86,KSC LC 39A,28.608058,-80.603956,1
87,KSC LC 39A,28.608058,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1
89,CCSFS SLC 40,28.561857,-80.577366,1


In [90]:
#Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.
marker_cluster = MarkerCluster()
marker_color = []
for i in spacex_df['Class']:
    if i == 1:
        marker_color.append('green')
    else:
        marker_color.append('red')
spacex_df['marker_color'] = marker_color
spacex_df.head()

/var/folders/vv/8slh1mqx7sddrb57ltbzgg0w0000gn/T/ipykernel_16901/152074201.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = marker_color


,LaunchSite,Latitude,Longitude,Class,marker_color
0,CCSFS SLC 40,28.561857,-80.577366,0,red
1,CCSFS SLC 40,28.561857,-80.577366,0,red
2,CCSFS SLC 40,28.561857,-80.577366,0,red
3,VAFB SLC 4E,34.632093,-120.610829,0,red
4,CCSFS SLC 40,28.561857,-80.577366,0,red


In [99]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker([record[1],record[2]],
                           icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

/var/folders/vv/8slh1mqx7sddrb57ltbzgg0w0000gn/T/ipykernel_16901/590216025.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marker = folium.Marker([record[1],record[2]],


In [98]:
for index, record in spacex_df.iterrows():
    print(index)
    print(record)
    print(record['marker_color'])

0
LaunchSite      CCSFS SLC 40
Latitude           28.561857
Longitude         -80.577366
Class                      0
marker_color             red
Name: 0, dtype: object
red
1
LaunchSite      CCSFS SLC 40
Latitude           28.561857
Longitude         -80.577366
Class                      0
marker_color             red
Name: 1, dtype: object
red
2
LaunchSite      CCSFS SLC 40
Latitude           28.561857
Longitude         -80.577366
Class                      0
marker_color             red
Name: 2, dtype: object
red
3
LaunchSite      VAFB SLC 4E
Latitude          34.632093
Longitude       -120.610829
Class                     0
marker_color            red
Name: 3, dtype: object
red
4
LaunchSite      CCSFS SLC 40
Latitude           28.561857
Longitude         -80.577366
Class                      0
marker_color             red
Name: 4, dtype: object
red
5
LaunchSite      CCSFS SLC 40
Latitude           28.561857
Longitude         -80.577366
Class                      0
marker_color     